In [ ]:
pip install ultralytics

In [ ]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow  # Import for Colab

# Load YOLOv8 model (you can use yolov8n, yolov8s, yolov8m, etc.)
model = YOLO('yolov8n.pt')

# Load the image
image_path = '/content/crowd_img.jpg'
image = cv2.imread(image_path)

# Perform detection on the image
results = model(image)

# Filter out only 'person' detections (Class 0 in COCO dataset)
person_count = 0
for result in results:
    for box in result.boxes:
        # Get class ID and confidence
        class_id = int(box.cls[0])
        if class_id == 0:  # Class '0' is 'person' in YOLO
            person_count += 1

print(f'Number of people detected: {person_count}')

# Optionally display the image with detected people
annotated_image = results[0].plot()
cv2_imshow(annotated_image)  # Use cv2_imshow for displaying in Colab


In [ ]:
from ultralytics import YOLO
import cv2
from IPython.display import HTML
from base64 import b64encode

model = YOLO('yolov8n.pt')

# Video path (Input)
video_path = '/content/bbbb.mp4'

# Video path (Output)
output_video_path = '/content/output_with_detections.mp4'

# Initialize video capture and video writer
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Define video codec and create VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)  # Get FPS from the input video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Real-time processing and saving to output video
ppl_threshold = 15  # Overcrowding threshold
overcrowding_alert = False  # Flag to track if overcrowding occurs in any frame
frame_count = 0  # Frame counter

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break  # End of video

    # Perform detection using YOLOv8
    results = model(frame)

    # Filter for 'person' detections and count
    try:
        person_count = sum(1 for result in results for box in result.boxes if int(box.cls[0]) == 0)
    except TypeError as e:
        print("Error in counting persons:", e)
        person_count = 0  # Fallback in case of error

    # Trigger overcrowding alert if people count exceeds the threshold
    if person_count > ppl_threshold:
        overcrowding_alert = True
        print(f"Overcrowding alert! Frame {frame_count} has {person_count} people.")

    # Annotate the frame with detections
    annotated_frame = results[0].plot()

    # Save the current annotated frame to the output video
    out.write(annotated_frame)

    frame_count += 1

# Release video resources
cap.release()
out.release()

# Output summary
if overcrowding_alert:
    print(f"Overcrowding occurred in at least one frame (Threshold: {ppl_threshold} people).")
else:
    print(f"No overcrowding detected (Threshold: {ppl_threshold} people).")

# Display the saved video in the notebook (for Colab or Jupyter environments)
video_data = open(output_video_path, 'rb').read()
video_encoded = b64encode(video_data).decode()
video_tag = f'<video controls src="data:video/mp4;base64,{video_encoded}">'
HTML(video_tag)
